In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
from pyvis.network import Network

# Parse the XML file
file_path = r"C:\Users\StanleyChan\Downloads\Training Capability Dashboard_20240801.twb"
tree = ET.parse(file_path)
root = tree.getroot()


# Find the <datasources> element that is a direct child of <workbook>, because the <datasources> element will also apear under the table/ view
datasources_element = None
for child in root:
    if child.tag == 'datasources':
        datasources_element = child
        break

# Extracting the <column> & group elements
columns = []
groups = []
group_set = set()

if datasources_element is not None:
    for datasource in datasources_element.findall('datasource'):
        for column in datasource.findall('column'):
            col_info = {
                "datasource": datasource.get('caption'),
                "underlying_name": column.get('name'),
                "presented_name": column.get('caption'),
                "datatype": column.get('datatype'),
                "role": column.get('role'),
                "type": column.get('type'),
                "class": None,
                "underlying_formula": None
            }

            # If Datasource is parameter then use 'name' attribute instead of 'caption' 
            if datasource.get('name') == 'Parameters':
                col_info["datasource"] = datasource.get('name')
                
            # Check if there is a <calculation> child element
            calculation = column.find('calculation')
            if calculation is not None:
                col_info["class"] = calculation.get('class')
                col_info["underlying_formula"] = calculation.get('formula')
            
            # Classify the column
            if column.get('param-domain-type') is not None:
                col_info["classification"] = "parameter"
            elif calculation is not None:
                col_info["classification"] = "calculation field"
            else:
                col_info["classification"] = "sourced"
            
            columns.append(col_info)

            # if presented_name is none, get from Underlyiny name
            if col_info["presented_name"] is None:
                col_info["presented_name"] = col_info["underlying_name"].strip('[]')
    
    #extract the set in <group> tag
    for group in datasource.findall('group'):
        if group.get('{http://www.tableausoftware.com/xml/user}ui-builder') == 'filter-group':
            groupfilter = group.find('groupfilter')
            if groupfilter is not None:  # the set may mark source field as member or level
                underlying_formula = groupfilter.get("member") 
                if underlying_formula is None:
                    underlying_formula = groupfilter.get("level")
                
                group_key = (group.get('caption'), group.get('name'))  # Use a tuple to represent the unique key
                if group_key not in group_set:
                    group_set.add(group_key)
                    group_info = {
                        "type": "group",
                        "datasource": datasource.get('caption'),
                        "underlying_name": group.get('name'),
                        "presented_name": group.get('caption'),
                        "datatype": "set",
                        "underlying_formula": underlying_formula,
                        "classification": "calculation field"
                    }
            groups.append(group_info)
            
columns_df = pd.DataFrame(columns)
groups_df = pd.DataFrame(groups)

combined_df = pd.concat([columns_df,groups_df]).reset_index(drop=True)
combined_df = combined_df.fillna('')
combined_df


,datasource,underlying_name,presented_name,datatype,role,type,class,underlying_formula,classification
0,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[@High level hierarchy (copy)_770678486698475580],DD High level hierarchy Drill down bars,string,dimension,nominal,tableau,//Used for the drill down bars only\r\n\r\nIf ...,calculation field
1,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[AchievedTarget],Achieved Target,real,measure,quantitative,,,sourced
2,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[ActualCount],Actual Count,integer,measure,quantitative,,,sourced
3,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[ActualIndividualCount],Actual Individual Count,integer,measure,quantitative,,,sourced
4,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[Actual],Actual,real,measure,quantitative,,,sourced
...,...,...,...,...,...,...,...,...,...
711,E03_TRAINING_CAPABILITY_METRICS_DEV,[training area competency filter from paramete...,training area filter not from parameter,string,dimension,nominal,tableau,if [Parameters].[Parameter 6]='5 Core training...,calculation field
712,E03_TRAINING_CAPABILITY_METRICS_DEV,[user filter (copy)_1226386513909456911],user filter,boolean,dimension,nominal,tableau,if [Parameters].[Parameter 16]=TRUE then \r\n\...,calculation field
713,E03_TRAINING_CAPABILITY_METRICS_DEV,[Course Code Set],Course Code Set,set,,group,,[CourseCode],calculation field
714,E03_TRAINING_CAPABILITY_METRICS_DEV,[DD label for set 2],DD label for set 2,set,,group,,[Calculation_2304154189457125404],calculation field


In [2]:
groups_df

,type,datasource,underlying_name,presented_name,datatype,underlying_formula,classification
0,group,E03_TRAINING_CAPABILITY_METRICS_DEV,[Course Code Set],Course Code Set,set,[CourseCode],calculation field
1,group,E03_TRAINING_CAPABILITY_METRICS_DEV,[DD label for set 2],DD label for set 2,set,[Calculation_2304154189457125404],calculation field
2,group,E03_TRAINING_CAPABILITY_METRICS_DEV,[test Set],dzv - RLS blocker 2,set,[Calculation_2100084844136275971],calculation field


In [3]:
##### replacing underlying_name with presented_name in the underlying_formula
# Create a mapping from Underlying Name to Presented Name
mapping = dict(zip(combined_df['underlying_name'], combined_df['presented_name']))

# Function to replace underlying names in the formula with presented names
def update_formula(formula, mapping):
    for underlying_name, presented_name in mapping.items():
        # Remove brackets from underlying_name for replacement
        clean_name = underlying_name.strip('[]')
        # Replace all instances of the underlying name in the formula
        formula = formula.replace(clean_name, presented_name)
    return formula

# Apply the function to create a new column 'New Formula'
combined_df['presented_formula'] = combined_df['underlying_formula'].apply(lambda x: update_formula(x, mapping))

# Print the updated DataFrame
combined_df.to_csv(r"C:\Users\StanleyChan\Downloads\field_list.csv")
combined_df

,datasource,underlying_name,presented_name,datatype,role,type,class,underlying_formula,classification,presented_formula
0,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[@High level hierarchy (copy)_770678486698475580],DD High level hierarchy Drill down bars,string,dimension,nominal,tableau,//Used for the drill down bars only\r\n\r\nIf ...,calculation field,//Used for the drill down bars only\r\n\r\nIf ...
1,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[AchievedTarget],Achieved Target,real,measure,quantitative,,,sourced,
2,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[ActualCount],Actual Count,integer,measure,quantitative,,,sourced,
3,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[ActualIndividualCount],Actual Individual Count,integer,measure,quantitative,,,sourced,
4,E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01...,[Actual],Actual,real,measure,quantitative,,,sourced,
...,...,...,...,...,...,...,...,...,...,...
711,E03_TRAINING_CAPABILITY_METRICS_DEV,[training area competency filter from paramete...,training area filter not from parameter,string,dimension,nominal,tableau,if [Parameters].[Parameter 6]='5 Core training...,calculation field,if [Parameters].[dzv brief exec summary]='5 Co...
712,E03_TRAINING_CAPABILITY_METRICS_DEV,[user filter (copy)_1226386513909456911],user filter,boolean,dimension,nominal,tableau,if [Parameters].[Parameter 16]=TRUE then \r\n\...,calculation field,if [Parameters].[RLS Activation]=TRUE then \r\...
713,E03_TRAINING_CAPABILITY_METRICS_DEV,[Course Code Set],Course Code Set,set,,group,,[CourseCode],calculation field,[Course Code]
714,E03_TRAINING_CAPABILITY_METRICS_DEV,[DD label for set 2],DD label for set 2,set,,group,,[Calculation_2304154189457125404],calculation field,[DD label for set]


In [4]:
# Get unique datasource values and remove 'Parameters'
datasources = [source for source in combined_df['datasource'].unique() if source != 'Parameters']
df_param = combined_df[combined_df['datasource'] == 'Parameters']

# Define the color map for classifications
color_map = {
    'parameter': '#936eb0',
    'sourced': 'green',
    'calculation field': '#f38c5f'
}

def generate_stats_html(df_filtered, source, color_map):
    stats_html = f"<h3>Statistics for datasource: {source}</h3>"
    stats_html += "<ul>"
    classification_counts = df_filtered['classification'].value_counts().to_dict()
    for classification, count in classification_counts.items():
        color = color_map.get(classification, 'black')  # Default to black if classification not in color_map
        stats_html += f"<li style='color: {color};'>{classification}: {count}</li>"
    stats_html += "</ul>"

    # Adding the additional lines of text at the end
    stats_html += """
    <p>    Noted that these graphs are not suitable for determining if the fields are redundant, 
    since independent fields could be used for actions and sheets.</p>
    <p>    The best use of this graph is to keep track of the interconnectivity of the fields and 
    estimate what would be effected when a field is subjected to change. It is also useful to 
    see the formula of the calculation field without opening each field.</p>
    """
    
    return stats_html

# Generate one file for each data source
for source in datasources:
    
    # Create a Network
    net = Network(notebook=True, cdn_resources='remote',height='750px', width='100%',select_menu=True,filter_menu=True,directed=True)
    # net.show_buttons(filter_=['physics']) # doesn't work if set_options is used
    
    net.set_options("""
        const options = {
        "physics": {
            "enabled": true,
            "forceAtlas2Based": {
            "springLength": 100
            },
            "minVelocity": 0.75,
            "solver": "forceAtlas2Based"
        }
    }""")   

    # Filter the DataFrame by the current datasource
    df_filtered = combined_df[combined_df['datasource'] == source]
    df_filtered = pd.concat([df_filtered, df_param])

    # Initialize a dictionary to keep track of node degrees
    node_degrees = {row['presented_name']: 0 for _, row in df_filtered.iterrows()}

    # Add nodes
    for _, row in df_filtered.iterrows():
        node_info = (
            f"Name: {row['presented_name']}\n"
            f"Datatype: {row['datatype']}\n"
            f"Type: {row['type']}\n"
            f"Classification: {row['classification']}\n"
            f"Datasource: {row['datasource']}\n\n"
        )

        if row['presented_formula']:
            node_info += (
                f"Formula:\n"
                f"{row['presented_formula']}"
            )
        # Assign a color based on the classification
        node_color = color_map.get(row['classification'], 'lightgray')
        net.add_node(row['presented_name'], label=row['presented_name'], title=node_info, color=node_color)
        
    # Add edges with arrows and count node degrees
    for _, row in df_filtered.iterrows():
        for _, row_target in df_filtered.iterrows():
            if row['underlying_name'] in row_target['underlying_formula']:
                net.add_edge(row['presented_name'], row_target['presented_name'], arrows='to')
                node_degrees[row['presented_name']] += 1
                node_degrees[row_target['presented_name']] += 1
 
    
    # Calculate the maximum node degree and update node sizes
    max_degree = max(node_degrees.values(), default=1)  # Ensure max_degree is at least 1 to avoid ZeroDivisionError
    for node_id, degree in node_degrees.items():
        # Use default size if all degrees are zero
        size = 10 if max_degree == 0 else 10 + (degree / max_degree) * 30
        net.get_node(node_id)['size'] = size

####### Add HTML header
    # Generate statistics HTML
    stats_html = generate_stats_html(df_filtered, source, color_map)
    
    # Save to HTML file
    file_name = f'network_{source}.html'
    net.write_html(file_name)

    # Read the existing HTML content
    with open(file_name, 'r') as f:
        content = f.read()

    # Insert the statistics HTML at the top of the body
    content = content.replace('<body>', f'<body>{stats_html}', 1)

    # Write the modified content back to the file
    with open(file_name, 'w') as f:
        f.write(content)

    print(f"Network graph for datasource '{source}' saved to {file_name}")

Network graph for datasource 'E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01_VIEWS_01)' saved to network_E03_TRAINING_CAPABILITY_METRICS_PROD+ (ERA_E01_VIEWS_01).html
Network graph for datasource 'E03_TRAINING_CAPABILITY_METRICS_DEV' saved to network_E03_TRAINING_CAPABILITY_METRICS_DEV.html
